In [1]:
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Image3Pipeline
import jwst.associations
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

from glob import glob
import astropy.io.fits as fits

import os
import astroquery
from astroquery.mast import Observations
from crds import client
import pandas as pd

from astropy.table import Table
from astropy.wcs import WCS
import numpy as np

In [2]:
os.environ['CRDS_PATH'] = '../CRDS/crds_cache/'
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
os.environ["CRDS_CONTEXT"] = "jwst_1188.pmap"

In [3]:
client.set_crds_server("https://jwst-crds.stsci.edu")

In [4]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

# figure sizes and resolutions
rcParams['figure.figsize'] = [11,8]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

In [5]:
import jwst
print(jwst.__version__)

1.12.5


# **Data**

In [ ]:
mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '../data/stage1/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

ext = 'rate'
# JWST images to be analyzed
image_files = []

for i in range(1,5):
    for j in range(1,5):
        for k in [3,4]:
            for l in ['a','b']:
                image_files.append( f"jw0178390400{k}_02101_0000{i}_nrc{l}{j}_{ext}.fits")

for image_file in image_files:
    # Download file (if not already downloaded)
    mast_path  = os.path.join(mast_dir, image_file)
    local_path = os.path.join(data_dir, image_file)
    Observations.download_file(mast_path, local_path=local_path)

# **Stage 2**

In [ ]:
def stage2(filename, output_dir):
    # Instantiate the pipeline
    img2 = Image2Pipeline()
    # Specify where the output should go
    img2.output_dir = output_dir
    # Save the final resulting _rate.fits files
    img2.save_results = True
    # Run the pipeline on an input list of files
    img2(filename)

In [ ]:
rate_files = glob(os.path.join("../data/stage1/", '*904003*nrc*_rate.fits'))
rate_files += glob(os.path.join("../data/stage1/", '*904004*nrc*_rate.fits'))

In [ ]:
len(rate_files)

In [ ]:
for f in rate_files:
    o = f.replace('stage1','stage2')
    o = o.replace('rate','cal')
    if not os.path.exists(o):
        stage2(f,"../data/stage2/")

In [ ]:
f

# **Stage 3**

In [ ]:
def stage3(filename, output_dir):
    # Instantiate the pipeline
    img3 = Image3Pipeline()
    # Specify where the output should go
    img3.output_dir = output_dir
    # Save the final resulting _rate.fits files
    img3.save_results = True
    # Run the pipeline on an input list of files
    img3(filename)

In [ ]:
cal_files = glob(os.path.join("../data/stage2/3004/", '*4003*nrc*_cal.fits'))
cal_files += glob(os.path.join("../data/stage2/3004/", '*4004*nrc*_cal.fits'))

In [ ]:
len(cal_files)

# **DOLPHOT**

In [ ]:
os.system(f"nircammask ../data/stage2/f115w_i2d.fits")

In [20]:
len(x)

551

In [ ]:
for m in ['a','b']:
    for n in range(1,5):
        
        # _CAL Files
        cal_files = glob(os.path.join("../data/stage2/3004/", f'*4003*nrc{m}{n}_cal.fits'))
        cal_files += glob(os.path.join("../data/stage2/3004/", f'*4004*nrc{m}{n}_cal.fits'))
        
        det_dir = f'../data/stage2/f115w/{m}{n}'
        if not os.path.exists(det_dir):
            os.mkdir(det_dir)
        
        if len(cal_files)!=8:
            print("File Error!")
            break
        # Generating directories
        exps = []
        for i,f in enumerate(cal_files):
            out_dir = f.split('/')[-1].split('.')[0]
            
            if not os.path.exists(f'{det_dir}/{out_dir}'):
                os.mkdir(f'{det_dir}/{out_dir}')
            if not os.path.exists(f"{det_dir}/{out_dir}/data.fits"):
                os.system(f"cp {f} {det_dir}/{out_dir}/data.fits")
                                  
            exps.append(f'{det_dir}/{out_dir}')
        
        # Applying NIRCAM Mask
        for f in exps:
            if not os.path.exists(f"{f}/data.sky.fits"):
                os.system(f"nircammask {f}/data.fits")
                os.system(f"calcsky {f}/data 10 25 2 2.25 2.00")
        
        # Completeness
        yy,xx = np.mgrid[100:3000:100,100:2000:100]
        x,y = xx.ravel(), yy.ravel()
        for m in range(20,30):
  
            mag = m + x*0
            ext = 1 + x*0
            chip = 1+ x*0

            df = pd.DataFrame(zip(ext,chip,x,y,mag), columns=['ext', 'chip', 'x', 'y', 'mag'])
            df.to_csv('../data/stage2/f115w_fake.txt',sep=' ', index=None,header=None)

            # Preparing Parameter file DOLPHOT NIRCAM
            with open("../params/nircam_dolphot_f115w.param") as f:
                        dat = f.readlines()

            dat[0] = f'Nimg = {len(exps)}                #number of images (int)\n'
            dat[4] = 'img0_file = ../data/stage2/f115w_i2d\n'

            dat[63] = f'FakeOut =     ../data/stage2/phot{m}.fake\n'

            for i,f in enumerate(exps):
                dat[5+i] = f'img{i+1}_file = {f}/data           #image {i+1}\n'

            with open("../params/nircam_dolphot_f115w.param", 'w', encoding='utf-8') as f:
                f.writelines(dat)

            # Running DOLPHOT NIRCAM
            os.system(f"dolphot {det_dir}/out -p../params/nircam_dolphot_f115w.param")
            
            os.system(f"python ../scripts/to_table_fake.py --o fake_out_{m} --n {len(exps)} --f ../data/stage2/phot{m}.fake")
        """
        # Generating Astropy FITS Table
        det= f'nrc{m}{n}'
        os.system(f"python ../scripts/to_table.py --o F115W_photometry_{det} --n {len(exps)} --f {det_dir}/out")

        phot_table = Table.read(f"{det_dir}/F115W_photometry_{det}.fits")
        phot_table.rename_columns(['mag_vega'],[f'mag_vega_F115W'])
        
        # Assingning RA-Dec using reference image
        hdu = fits.open(f"../data/stage2/F115W_i2d.fits")[1]

        wcs = WCS(hdu.header)
        positions = np.transpose([phot_table['x'] - 0.5, phot_table['y']-0.5])

        coords = np.array(wcs.pixel_to_world_values(positions))

        phot_table['ra']  = coords[:,0]
        phot_table['dec'] = coords[:,1]
        
        # Filtering stellar photometry catalog using Warfield et.al (2023)
        phot_table1 = phot_table[ (phot_table['sharpness']**2   <= 0.01) &
                                    (phot_table['obj_crowd']    <=  0.5) &
                                    (phot_table['flags']        <=    2) &
                                    (phot_table['type']         <=    2)]

        phot_table2 = phot_table[ ~((phot_table['sharpness']**2 <= 0.01) &
                                    (phot_table['obj_crowd']    <=  0.5) &
                                    (phot_table['flags']        <=    2) &
                                    (phot_table['type']         <=    2))]
        print('NIRCAM SHORT')
        phot_table1.write(f'{det_dir}/F115W_photometry_{det}_filt.fits', overwrite=True)
        """
        break
    break

Reading FITS file ../data/stage2/f115w/a1/jw01783004003_02101_00001_nrca1_cal/data.fits
Reading FITS file ../data/stage2/f115w/a1/jw01783004003_02101_00002_nrca1_cal/data.fits
Reading FITS file ../data/stage2/f115w/a1/jw01783004003_02101_00003_nrca1_cal/data.fits
Reading FITS file ../data/stage2/f115w/a1/jw01783004003_02101_00004_nrca1_cal/data.fits
Reading FITS file ../data/stage2/f115w/a1/jw01783004004_02101_00001_nrca1_cal/data.fits
Reading FITS file ../data/stage2/f115w/a1/jw01783004004_02101_00002_nrca1_cal/data.fits
Reading FITS file ../data/stage2/f115w/a1/jw01783004004_02101_00003_nrca1_cal/data.fits
Reading FITS file ../data/stage2/f115w/a1/jw01783004004_02101_00004_nrca1_cal/data.fits
Reading FITS file ../data/stage2/f115w_i2d.fits
Reading IMAGE extension: 2048x2048
  GAIN=2.08 EXP=365s NOISE=11.43 BAD=-793.77 SAT=310333.81
Reading IMAGE extension: 2048x2048
  GAIN=2.08 EXP=365s NOISE=11.43 BAD=-716.79 SAT=488588.47
Reading IMAGE extension: 2048x2048
  GAIN=2.08 EXP=365s NOIS

In [22]:
os.system(f"python ../scripts/to_table_fake.py --o fake_out --n {len(exps)} --f ../data/stage2/phot.fake")

0

In [25]:
phot_table  = Table.read(f'../data/stage2/fake_out.fits')

In [26]:
phot_table1 = phot_table[ (phot_table['sharpness']**2   <= 0.01) &
                                    (phot_table['obj_crowd']    <=  0.5) &
                                    (phot_table['flags']        <=    2) &
                                    (phot_table['type']         <=    2)]

In [28]:
len(phot_table1)

418

In [27]:
phot_table1

ext_inp,chip_inp,x_inp,y_inp,chi_fit_inp,obj_SNR_inp,obj_sharpness_inp,obj_roundness_inp,dir_maj_axis_inp,obj_crowd_inp,type_inp,counts_tot_inp,sky_tot_inp,count_rate_inp,count_rate_err_inp,mag_vega_inp,mag_ubvri_inp,mag_err_inp,chi_inp,SNR_inp,ext,chip,x,y,chi_fit,obj_SNR,obj_sharpness,obj_roundness,dir_maj_axis,obj_crowd,type,counts_tot,sky_tot,count_rate,count_rate_err,mag_vega,mag_ubvri,mag_err,chi,SNR,sharpness,roundness,crowd,flags
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1.0,1.0,100.0,100.0,90918.3,20.0,90904.7,20.0,91187.2,20.0,90863.4,20.0,90897.8,20.0,90866.4,20.0,90887.9,20.0,90921.0,20.0,1.0,1.0,100.01,100.0,0.27,513.2,-0.003,0.003,10.0,0.0,1.0,730866.4,466.25,1.01e-08,1.96e-11,19.994,99.999,0.002,0.27,513.2,-0.003,0.003,0.0,0.0
1.0,1.0,200.0,100.0,90918.3,20.0,90904.7,20.0,91187.2,20.0,90863.4,20.0,90897.8,20.0,90866.4,20.0,90887.9,20.0,90921.0,20.0,1.0,1.0,200.0,100.01,0.41,512.0,-0.004,0.002,120.0,0.004,1.0,731852.2,475.18,1.01e-08,1.97e-11,19.993,99.999,0.002,0.41,512.0,-0.004,0.002,0.004,0.0
1.0,1.0,300.0,100.0,90918.3,20.0,90904.7,20.0,91187.2,20.0,90863.4,20.0,90897.8,20.0,90866.4,20.0,90887.9,20.0,90921.0,20.0,1.0,1.0,299.98,100.0,0.36,510.7,-0.002,0.01,0.0,0.0,1.0,729134.8,487.47,1e-08,1.96e-11,19.997,99.999,0.002,0.36,510.7,-0.002,0.01,0.0,0.0
1.0,1.0,400.0,100.0,90918.3,20.0,90904.7,20.0,91187.2,20.0,90863.4,20.0,90897.8,20.0,90866.4,20.0,90887.9,20.0,90921.0,20.0,1.0,1.0,400.0,100.0,0.27,514.0,-0.004,0.004,160.0,0.0,1.0,732972.4,523.83,1.01e-08,1.96e-11,19.991,99.999,0.002,0.27,514.0,-0.004,0.004,0.0,0.0
1.0,1.0,500.0,100.0,90918.3,20.0,90904.7,20.0,91187.2,20.0,90863.4,20.0,90897.8,20.0,90866.4,20.0,90887.9,20.0,90921.0,20.0,1.0,1.0,500.0,100.0,0.35,512.4,-0.006,0.0,80.0,0.001,1.0,728087.6,498.21,1e-08,1.95e-11,19.998,99.999,0.002,0.35,512.4,-0.006,0.0,0.001,0.0
1.0,1.0,600.0,100.0,90918.3,20.0,90904.7,20.0,91187.2,20.0,90863.4,20.0,90897.8,20.0,90866.4,20.0,90887.9,20.0,90921.0,20.0,1.0,1.0,600.0,100.0,0.32,513.6,-0.001,0.001,0.0,0.0,1.0,730920.0,493.11,1.01e-08,1.96e-11,19.994,99.999,0.002,0.32,513.6,-0.001,0.001,0.0,0.0
1.0,1.0,700.0,100.0,90918.3,20.0,90904.7,20.0,91187.2,20.0,90863.4,20.0,90897.8,20.0,90866.4,20.0,90887.9,20.0,90921.0,20.0,1.0,1.0,700.0,100.0,0.26,513.8,-0.001,0.002,30.0,0.0,1.0,730959.2,490.67,1.01e-08,1.96e-11,19.994,99.999,0.002,0.26,513.8,-0.001,0.002,0.0,0.0
1.0,1.0,800.0,100.0,90918.3,20.0,90904.7,20.0,91187.2,20.0,90863.4,20.0,90897.8,20.0,90866.4,20.0,90887.9,20.0,90921.0,20.0,1.0,1.0,800.0,100.0,0.26,513.4,-0.004,0.002,105.0,0.0,1.0,731266.7,491.65,1.01e-08,1.96e-11,19.994,99.999,0.002,0.26,513.4,-0.004,0.002,0.0,0.0
1.0,1.0,900.0,100.0,90918.3,20.0,90904.7,20.0,91187.2,20.0,90863.4,20.0,90897.8,20.0,90866.4,20.0,90887.9,20.0,90921.0,20.0,1.0,1.0,900.0,100.0,0.26,513.0,-0.002,0.005,45.0,0.0,1.0,729669.9,496.08,1e-08,1.96e-11,19.996,99.999,0.002,0.26,513.0,-0.002,0.005,0.0,0.0


'FakeOut =     ../data/stage2/phot.fake         #file with fake star output data (default=phot.fake)\n'